In [1]:
import rasterio
from rasterio.features import shapes
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
from fiona.crs import from_epsg
from matplotlib.colors import LightSource
import geopandas as gpd
import requests
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import open3d as o3d
import matplotlib.pyplot as plt
import seaborn as sns
import fiona
import pycrs
import json
import pprint
import plotly.graph_objs as go
import plotly.express as px
from datetime import datetime

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATAS =  {'LGE' : {'MNT': '/media/bestycame/407C5CD57C5CC776/3D/RELIEF_WALLONIE_MNT_2013_2014_LIEGE.tif',
                   'MNS': '/media/bestycame/407C5CD57C5CC776/3D/RELIEF_WALLONIE_MNS_2013_2014_LIEGE.tif'},
          'HAI' : {'MNT': '/mnt/3D/RELIEF_WALLONIE_MNT_2013_2014_HAINAUT.tif',
                   'MNS': '/mnt/3D/RELIEF_WALLONIE_MNS_2013_2014_HAINAUT.tif'},
          'LUX' : {'MNT': '/mnt/3D/RELIEF_WALLONIE_MNT_2013_2014_LUXEMBOURG.tif',
                   'MNS': '/mnt/3D/RELIEF_WALLONIE_MNS_2013_2014_LUXEMBOURG.tif'},
          'NAM' : {'MNT': '/mnt/3D/RELIEF_WALLONIE_MNT_2013_2014_NAMUR.tif',
                   'MNS': '/mnt/3D/RELIEF_WALLONIE_MNS_2013_2014_NAMUR.tif'},
          'BRA' : {'MNT': '/mnt/3D/RELIEF_WALLONIE_MNT_2013_2014_BRABANT.tif',
                   'MNS': '/mnt/3D/RELIEF_WALLONIE_MNS_2013_2014_BRABANT.tif'}}
BATI_3D = '/media/bestycame/407C5CD57C5CC776/3D/BATI3D_2013-2014.gdb'

In [3]:
# gdal_calc.py -A DSM/GeoTIFF/DHMVII_DSM_1m_testdata.tif -B ./dtm_1m.tif --outfile=CHM.tif --calc="A-B" --NoDataValue=0
# gdalwarp  DTM/GeoTIFF/DHMVII_DTM_5m_testdata.tif test.tif -tr 1 1 -r near
# gdal_contour -3d -a elev chm.tif contour.shp -i 0.5 -snodata 0b

In [10]:
gdb = fiona.open(BATI_3D, layer=1)
hits = list(gdb.items(bbox=(xMin, yMin, xMax, yMax)))
meshes = []
for hit in hits:
    points = hit[1]['geometry']['coordinates']
    pts = [item for sublist in points for sitem in sublist for item in sitem]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)
    mesh = pcd.compute_convex_hull()
    meshes.append(mesh[0])
o3d.visualization.draw_geometries(meshes, point_show_normal=True, mesh_show_wireframe=True, mesh_show_back_face=True)

NameError: name 'xMin' is not defined

In [4]:
# Christophe Schenke
# christophe.shanke@spw.walonnie.be  
# 0475 95 4570

In [4]:
from pyproj import Transformer
def lambert_to_wgs(x_lambert, y_lambert):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326")
    x_wgs, y_wgs = transformer.transform(x_lambert, y_lambert)
    return x_wgs, y_wgs

def wgs_to_lambert(x_wgs, y_wgs):
    transformer = Transformer.from_crs("epsg:4326", "epsg:31370")
    x_lambert, y_lambert = transformer.transform(x_wgs, y_wgs)
    return x_lambert, y_lambert

# x, y, xMin, xMax, yMin, yMax = search_address(cp='5000', rue='RUE DE LA CRETE', num = '111', as_wgs=False)
def search_address(cp, rue, num, as_wgs=False, as_dict=False, boundary=False):    
    url = 'http://geoservices.wallonie.be/geolocalisation/rest/getPositionByCpRueAndNumero/{cp}/{rue}/{num}'
    r = requests.get(url.format(cp=cp, rue=rue, num=num)).json()
    x, y, xMin, xMax, yMin, yMax = r['x'], r['y'], r['rue']['xMin'], r['rue']['xMax'], r['rue']['yMin'], r['rue']['yMax']
    if as_wgs:
        x, y       = lambert_to_wgs(x, y)
        xMin, xMax = lambert_to_wgs(xMin, xMax)
        yMin, yMax = lambert_to_wgs(yMin, yMax)
    if as_dict:
        return {'x': x, 'xMin': xMin, 'xMax': xMax, 
                'y': y, 'yMin': yMin, 'yMax': yMax}
    return x, y, xMin, xMax, yMin, yMax

#x, y, xMin, xMax, yMin, yMax = search_address_mapbox("Rue du roton 38, 6000 Charleroi", as_wgs=False, boundary=150)
def search_address_mapbox(address, as_wgs=False, as_dict=False, boundary=100):
    key = 'pk.eyJ1IjoiYmFjYXlhdzU1OSIsImEiOiJja2dtNTgyNW8wMWN2MnBzM2loNGt4NmlzIn0.3IYulU7qzfpq2Ms0aaYWMQ'
    url = "https://api.mapbox.com/geocoding/v5/mapbox.places/{address}.json?types=address&access_token={key}"
    r = requests.get(url.format(address=address,key=key))
    if r.status_code != 200:
        return 'NotFound'
    r = r.json()
    y, x = r['features'][0]['center']
    x, y = wgs_to_lambert(x, y)
    xMin, xMax = x - boundary, x + boundary
    yMin, yMax = y - boundary, y + boundary
    if as_wgs:
        x, y       = lambert_to_wgs(x, y)
        xMin, xMax = lambert_to_wgs(xMin, xMax)
        yMin, yMax = lambert_to_wgs(yMin, yMax) 
    if as_dict:
        return {'x': x, 'xMin': xMin, 'xMax': xMax, 
                'y': y, 'yMin': yMin, 'yMax': yMax}
    return x, y, xMin, xMax, yMin, yMax

def is_in_bbox(x, y, bbox):
    xMin, yMin, xMax, yMax = bbox
    if xMin <= x <= xMax and  yMin <= y <= yMax:
        return True
    return False

def find_files(xMin, xMax, yMin, yMax):
    bboxes = {key:rasterio.open(next(iter(DATAS[key].values()))).bounds for key in DATAS.keys()}
    for k, bbox in bboxes.items():
        if is_in_bbox(xMin, yMin, bbox) and is_in_bbox(xMax, yMax, bbox):
            return DATAS[k]
    return 'File not found'

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

def subsetTif(xMin, xMax, yMin, yMax, in_tif, out_tif=None):
    if out_tif == None:
        out_tif = datetime.now().strftime('%Y%m%d_%H%M%S.%f') + '.tif'
    data = rasterio.open(in_tif)
    geo = gpd.GeoDataFrame({'geometry': box(xMin, yMin, xMax, yMax)}, index=[0], crs=from_epsg(31370))
    out_img, out_transform = mask(dataset=data, shapes=getFeatures(geo), filled=False, crop=True)
    out_meta = data.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_img.shape[1],
                 "width": out_img.shape[2],
                 "transform": out_transform,
                "crs": pycrs.parse.from_epsg_code(31370).to_proj4()
                    })
    with rasterio.open(out_tif, "w", **out_meta) as dest:
        dest.write(out_img)
    return out_tif

#show_tif('subset.tif', title=address, hillshade=False)
def show_tif(tif, title=None, hillshade=False):
    if title == None:
        title = tif
    colors = [[0.0, 'rgb(000, 063, 076)'], [0.1, 'rgb(029, 081, 059)'], [0.2, 'rgb(055, 098, 043)'], 
             [0.3, 'rgb(079, 114, 030)'], [0.4, 'rgb(103, 129, 016)'], [0.5, 'rgb(136, 142, 002)'], 
             [0.6, 'rgb(169, 154, 021)'], [0.7, 'rgb(192, 171, 045)'], [0.8, 'rgb(214, 188, 074)'],
             [0.9, 'rgb(234, 209, 112)'], [1.0, 'rgb(254, 229, 152)']]
    
    with rasterio.open(tif) as src:
        data = src.read(1)

    if hillshade:
        light = LightSource(azdeg=315, altdeg=45)
        data = light.hillshade(data, vert_exag=1)     

    fig = px.imshow(data, color_continuous_scale=colors)
    fig.update_traces(customdata=data, hovertemplate='elevation: %{customdata:.4f}')
    fig.update_layout(**dict(title_text=title, width=700, height=500, template='none'))
    return fig

def create_chm(xMin, xMax, yMin, yMax):
    files = find_files(xMin, xMax, yMin, yMax)
    time = datetime.now().strftime('%Y%m%d_%H%M%S.%f') + '.tif'
    for k, file in files.items():
        subsetTif(xMin, xMax, yMin, yMax, file, k +'_'+time)
    !gdal_calc.py -A MNS_$time -B MNT_$time --outfile=CHM_$time --calc="A-B" --NoDataValue -9999 
    return time

In [5]:
def _to_2d(x, y, z):
    return tuple(filter(None, [x, y]))

In [11]:
address = "Rue du Roton 38"
x, y, xMin, xMax, yMin, yMax = search_address_mapbox(address, as_wgs=False, boundary=50)
time = create_chm(xMin, xMax, yMin, yMax)

/home/bestycame/.local/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



0.. 16.. 33.. 50.. 66.. 83.. 100 - Done


In [12]:
gdb = fiona.open(BATI_3D, layer=0)
hits = list(gdb.items(bbox=(xMin, yMin, xMax, yMax)))

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
polygons = []

features = []

for hit in hits:
    features.append({'id':           hit[0],
                        'H_MUR':        hit[1]['properties']['H_MUR'],
                        'H_TOIT':       hit[1]['properties']['H_TOIT'],
                        'E_TOIT':       hit[1]['properties']['E_TOIT'],
                        'Q_LIDAR':      hit[1]['properties']['Q_LIDAR'],
                        'Q_BATI' :      hit[1]['properties']['Q_BATI'],
                        'SHAPE_Length': hit[1]['properties']['SHAPE_Length'],
                        'SHAPE_Area':   hit[1]['properties']['SHAPE_Area'],
                        'X': [int(x[0]) for x in hit[1]['geometry']['coordinates'][0][0]],
                        'Y': [int(x[1]) for x in hit[1]['geometry']['coordinates'][0][0]]
                       })
    polygons.append(Polygon([_to_2d(x, y, z) for x, y, z in hit[1]['geometry']['coordinates'][0][0]]))

In [13]:
data = rasterio.open(f'CHM_{time}')
xMin, yMin, xMax, yMax = data.bounds
z_data = pd.DataFrame(data.read(1))
z_data.index   = list(range(int(yMax), int(yMin), -1))
z_data.columns = list(range(int(xMin), int(xMax)))
z_data = z_data.applymap(lambda x: 0.5 if x < 1 else round(x, 1))

In [14]:
#z_data_old = z_data.copy()
#z_data[:] = 0
#for y in range(int(yMin) + 1, int(yMax)):
#    for x in range(int(xMin), int(xMax)):
#        point = Point(x, y)
#        for poly in polygons:
#            if poly.contains(point):
#                z_data.loc[y, x] = z_data_old.loc[y, x]

In [26]:
len(hits)

63

In [35]:
hits[-1][1]['geometry']['coordinates'][0][0]

[(155665.5169999972, 123550.94900000095, 119.86999999999534),
 (155682.09799999744, 123539.41299999878, 119.86999999999534),
 (155686.5930000022, 123538.1649999991, 119.86999999999534),
 (155683.73399999738, 123527.87099999934, 119.86999999999534),
 (155677.52600000054, 123529.59800000116, 119.86999999999534),
 (155649.53299999982, 123549.11699999869, 119.86999999999534),
 (155636.8669999987, 123530.57999999821, 119.86999999999534),
 (155633.21500000358, 123525.24700000137, 119.86999999999534),
 (155615.5099999979, 123499.78299999982, 119.86999999999534),
 (155610.4699999988, 123492.53400000185, 119.86999999999534),
 (155644.97299999744, 123468.54500000179, 119.86999999999534),
 (155641.78899999708, 123459.5, 119.86999999999534),
 (155605.11800000072, 123484.99700000137, 119.86999999999534),
 (155579.46500000358, 123448.09899999946, 119.86999999999534),
 (155619.50299999863, 123426.0, 119.86999999999534),
 (155621.36900000274, 123425.01000000164, 119.86999999999534),
 (155616.200000002

In [15]:
fig = go.Figure(go.Surface(z=z_data.values, x=z_data.columns, y=z_data.index, opacity=1))
         
for feature in features:
    fig.add_scatter3d(x=feature['X'], y=feature['Y'], z=[3]*len(feature['X']), 
                      mode='lines',showlegend=False, 
                      line=dict(color='red', width=10)
                     )

fig.update_layout(
    title=address,
    margin=dict(t=40, r=0, l=0, b=40),
    scene = {"xaxis": {'showspikes': False},
             "yaxis": {'showspikes': False},
             "zaxis": {'showspikes': False},
             'camera_eye': {"x": 0, "y": -0.5, "z": 0.5},
             "aspectratio": {"x": 1, "y": 1, "z": 0.1}
            })
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
#fig.write_html('./templates/map.html',
#                   full_html=False,
#                   include_plotlyjs='cdn')


**Samuel:**
  * BLENDER!!
  * Remove scatterplot outside boundary
  * Scatterplot above the surfaceplot
  * add cadastre lines in different colors
  * colors of the map: Water <0 ground/green < 1 < brown to orange?
  * add hover information
  * remove spikelines
  * add 'here' marker for x, y
 
 
**Olivier:**
  * Continue Docker CI CD
  * access tiff files over internet
  * cache of already generated graphs
 
  
  
**Sergei:**
  * improve speed of algorythm
  * implement tests
  * typing, docstrings